In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import datetime as dt
from technicals.indicators import RSI, MACD, ATR, BollingerBands
from technicals.patterns import apply_patterns
from simulation.guru_tester import GuruTester
from models.trade_settings import TradeSettings
from constants import defs
import json
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns


In [3]:
BUY = 1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0

def apply_signal(row):
    
    if (
            row.ENGULFING == True and
            row.direction == BUY and 
            row.mid_l > row.EMA_200 and
            row.RSI_14 > RSI_LIMIT and 
            row.SPREAD <= row.maxspread and
            row.GAIN >= row.mingain and
            row.mid_c < row.BB_LW and row.mid_o > row.BB_LW and  # Bollinger Bands condition for BUY
            row[f"RSI_{row.rsi_period}"] < row.rsi_oversold and  # RSI condition for oversold
            row.MACD > row.SIGNAL
         
        ):
        print(row)
        return BUY
    if (
            row.direction == SELL and row.mid_h < row.EMA_200 and
            row.RSI_14 < RSI_LIMIT and 
            row.SPREAD <= row.maxspread and
            row.GAIN >= row.mingain and
            row.mid_c > row.BB_UP and row.mid_o < row.BB_UP and  # Bollinger Bands condition for SELL
            row[f"RSI_{row.rsi_period}"] > row.rsi_overbought  and # RSI condition for overbought
            row.MACD < row.SIGNAL 
        ):
        print(row)
        return SELL
    
    return NONE        

In [4]:
def apply_SL(row ):
    """
    Calculates the Stop Loss (SL) using ATR for dynamic adjustment based on volatility.
    """
    if row.SIGNAL == defs.BUY:
        return row.mid_c - (row[f"ATR_{row.atr_period}"] * row.atr_multiplier)
    elif row.SIGNAL == defs.SELL:
        return row.mid_c + (row[f"ATR_{row.atr_period}"] * row.atr_multiplier)
    return 0.0


def apply_TP(row ):
    """
    Calculates the Take Profit (TP) using ATR and risk-reward ratio.
    """
    if row.SIGNAL == defs.BUY:
        return row.mid_c + (row[f"ATR_{row.atr_period}"] * row.atr_multiplier * row.riskreward)
    elif row.SIGNAL == defs.SELL:
        return row.mid_c - (row[f"ATR_{row.atr_period}"] * row.atr_multiplier * row.riskreward)
    return 0.0

In [5]:
df_an = pd.read_pickle(f"../data/EUR_USD_H1.pkl")
print(df_an.head())


                       time  volume    mid_o    mid_h    mid_l    mid_c  \
0 2020-01-01 22:00:00+00:00      74  1.12124  1.12162  1.12124  1.12152   
1 2020-01-01 23:00:00+00:00     266  1.12149  1.12224  1.12149  1.12189   
2 2020-01-02 00:00:00+00:00     114  1.12187  1.12192  1.12158  1.12184   
3 2020-01-02 01:00:00+00:00     219  1.12186  1.12246  1.12182  1.12209   
4 2020-01-02 02:00:00+00:00     137  1.12206  1.12247  1.12185  1.12224   

     bid_o    bid_h    bid_l    bid_c    ask_o    ask_h    ask_l    ask_c  
0  1.12083  1.12141  1.12075  1.12132  1.12165  1.12205  1.12160  1.12171  
1  1.12134  1.12210  1.12134  1.12182  1.12164  1.12260  1.12164  1.12196  
2  1.12180  1.12185  1.12152  1.12177  1.12194  1.12198  1.12165  1.12190  
3  1.12179  1.12239  1.12175  1.12203  1.12192  1.12253  1.12189  1.12215  
4  1.12200  1.12240  1.12177  1.12217  1.12213  1.12255  1.12191  1.12231  


In [9]:



def run_pair(pair):
        df_an = pd.read_pickle(f"../data/{pair}_H1.pkl")
        df_m5 = pd.read_pickle(f"../data/{pair}_M5.pkl")
        df_an.reset_index(drop=True, inplace=True)
        df_m5.reset_index(drop=True, inplace=True)
        with open("../bot/settings.json", "r") as f:
                data = json.loads(f.read())
                trade_settings = {k: TradeSettings(v, k) for k, v in data['pairs'].items()}
                trade_risk = data['trade_risk']
        pair_settings = trade_settings[pair]
        # print(pair_settings)
        # Access the TradeSettings for the given pair

        df_an['PAIR'] = pair
        df_an['SPREAD'] = df_an.ask_c - df_an.bid_c

        df_an = RSI(df_an)
        df_an = BollingerBands(df_an, pair_settings.n_ma, pair_settings.n_std)
        df_an = ATR(df_an, pair_settings.atr_period)
        df_an = RSI(df_an, pair_settings.rsi_period)
        df_an = MACD(df_an)
        df_an = apply_patterns(df_an)
        df_an['EMA_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean()
        df_an['GAIN'] = abs(df_an.mid_c - df_an.BB_MA)
        df_an['SIGNAL'] = df_an.apply(apply_signal, axis=1, trade_settings=pair_settings)
        df_an['SL'] = df_an.apply(apply_SL, axis=1, trade_settings=pair_settings)
        df_an['TP'] = df_an.apply(apply_TP, axis=1, trade_settings=pair_settings)
        df_an['LOSS'] = abs(df_an.mid_c - df_an.SL)
        df_an['TRADE_RISK'] = trade_risk
        df_an['n_ma'] = pair_settings.n_ma
        df_an['n_std'] = pair_settings.n_std
        df_an['maxspread']= pair_settings.maxspread
        df_an['mingain']= pair_settings.mingain
        df_an['riskreward']= pair_settings.riskreward
        df_an['atr_period'] = pair_settings.atr_period
        df_an['atr_multiplier']= pair_settings.atr_multiplier
        df_an['atr_threshold']= pair_settings.atr_threshold
        df_an['rsi_period']= pair_settings.rsi_period
        df_an['rsi_overbought']= pair_settings.rsi_overbought
        df_an['rsi_oversold']= pair_settings.rsi_oversold


        our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
        'bid_o', 'bid_h', 'bid_l', 'bid_c', 
        'ask_o', 'ask_h', 'ask_l', 'ask_c',
        'ENGULFING', 'direction', 'EMA_200','MACD',
        'SIGNAL_MD','HIST','RSI_14','ATR_14', 'BB_UP', 'BB_LW', 
        'BB_MA','GAIN','SIGNAL','SL','TP','LOSS','SPREAD','PAIR',
        'TRADE_RISK','n_ma','n_std','maxspread','mingain','riskreward',
        'atr_period','atr_multiplier','atr_threshold','rsi_period','rsi_overbought',
        'rsi_oversold']


        df_slim = df_an[our_cols].copy()
        df_slim.dropna(inplace=True)

        df_slim.reset_index(drop=True, inplace=True)
        print(df_slim.head())

        # gt = GuruTester(
        # df_slim,
        # apply_signal,  
        # df_m5,
        # use_spread=True
        # )

        # gt.run_test()
        # return gt.df_results

    

In [8]:
run_pair("EUR_USD")

TypeError: apply_signal() got an unexpected keyword argument 'trade_settings'

In [ ]:
res = []
for p in [ "EUR_USD", "GBP_JPY" ]:
    res.append(dict(pair=p,res=run_pair(p)))

In [ ]:
for r in res:
    print(r['pair'], r['res'].result.sum())